In [1]:
import numpy as np
import matplotlib.pyplot as plt

X = (n'd)

Y = (n'1)

w = (d'1)

b = (n'1)

argmin sum(X.w+b-y)^2 + penalty.sum(abs(y))

In [2]:
dim_n = 500
dim_d = 1000
dim_k = 100
sig = 1

In [99]:
def lasso_coordinate_descent(X,y,w0,penalty,tol, debug=False):
    n,d = X.shape
    a = np.zeros(d)
    c = np.zeros(d)
    w_new = np.copy(w0)
    
    gradient_descending = True
    while gradient_descending:
        w_old = np.copy(w_new)
        b = (1/n)*np.sum(y - X.dot(w_new))  
        for k in range(d):
            a[k] = 2*np.sum(np.power(X[:,k],2))
            c[k] = np.sum(X[:,k]*(y - (b + X.dot(w_new) - X[:,k]*w_new[k])))
            
            if c[k] > penalty:
                w_new[k] = (c[k] - penalty)/a[k]
            elif c[k] < -penalty:
                w_new[k] = (c[k] + penalty)/a[k]
            else:
                w_new[k] = 0
                
        if debug:
            obj = compute_objective(X, y, w_new, b, penalty)
            print('obj: {}, w: {}'.format(obj, np.max(np.abs(w_new - w_old))))
        # ... if you're not moving more than tol in any component, exit
        gradient_descending = np.max(np.abs(w_new - w_old)) > tol
    return w_new,b

In [62]:
 def update_w(a,c,penalty):
    d, = a.shape
    w_new = np.zeros(d)
    for k in range(d):  
#         if c[k] < -penalty:
#             w_new[k] = (c[k]+penalty)/a[k]
#         elif (-penalty <= c[k]) and (c[k] <= penalty):
#             w_new[k] = 0
#         else:
#             w_new[k] = (c[k]-penalty)/a[k]
        if c[k] > penalty:
            w_new[k] = (c[k] - penalty)/a[k]
        elif c[k] < -penalty:
            w_new[k] = (c[k] + penalty)/a[k]
        else:
            w_new[k] = 0
    return w_new

In [18]:
def compute_a_c_1(X, y, w, b):
    n,d = X.shape
    
    # 1'd: sum along squared columns
    a = 2*np.sum(np.square(X),axis=0) 
    
    # Xw_skip n'd: skipping kth col
    Xw_skip = np.ma.dot(X, np.ma.array(np.tile(w,(d,1)).T, mask=np.identity(d))).data
    c = 2*(np.einsum('nd,n->d', X_data, y_data-b)-np.einsum('nd,nd->d',X_data,Xw_skip))
    
    return a,c

In [47]:
def compute_a_c_2(X, y, w, b):
    n,d = X.shape

    a = np.zeros(d)
    c = np.zeros(d)
    for k in range(d):
        a[k] = 2*np.sum(np.square(X[:,k]))
        c[k] = np.sum(X[:,k]*(y-(b+X.dot(w)-X[:,k]*w[k])))
        
    return a,c

In [48]:
#             c[j] = np.sum(X[:,j]*(y - (b + X.dot(w) - X[:,j]*w[j])))

In [49]:
def compute_objective(X, y, w, b, penalty):
    return np.sum(np.square(X@w+b-y)) + penalty*np.sum(np.abs(w))

In [57]:
def coordinate_descent(X, y, w, penalty, delta_stop, max_iterations):
    iterations=0
    w_old = w.copy()
    w_new = w.copy()
    while iterations < max_iterations:
        
        b = update_b(X,y,w)
        w_old = w_new.copy()
        a,c = compute_a_c_2(X, y, w_new, b) # d'1
        w_new = update_w(a, c, penalty)
        
        # Visual sanity check on objective!
        objective_value = compute_objective(X,y,w_new,b,penalty)
       
        
        # Exit condition
        if np.max(np.abs(w_new - w_old)) < delta_stop:
            print('Success!')
            break
        
        iterations=iterations+1
    return w_new, b

In [58]:
# This sets up the data!

np.random.seed(1)
X_data = np.random.randn(dim_n, dim_d)
eps = sig*np.random.randn(dim_n)

w_exact = np.array([j/dim_k if j < dim_k else 0 for j in range(1,dim_d+1)])
y_data = np.dot(X_data,w_exact)+eps

w0 = np.random.rand(dim_d)
b = update_b(X_data,y_data, w0)

In [59]:
# atest1 = 2*np.sum(np.square(X_data),axis=0)
# Xw_skip = np.ma.dot(X_data, np.ma.array(np.tile(w0,(dim_d,1)).T, mask=np.identity(dim_d))).data
# X_Xw_skip = np.einsum('ij,ji->i', X_data.T, Xw_skip)
# ctest1 = 2*(X_data.T@(y_data-b) - X_Xw_skip)

In [53]:
# Test!
a1,c1 = compute_a_c_1(X_data, y_data, w0, b)
a2,c2 = compute_a_c_2(X_data, y_data, w0, b)
print(np.all(np.abs(a1-a2)<1e-10))
print(np.all(np.abs(c1-c2)<1e-10))

a1,c1 = compute_a_c_1(X_data, y_data, w_exact, b)
a2,c2 = compute_a_c_2(X_data, y_data, w_exact, b)
print(np.all(np.abs(a1-a2)<1e-10))
print(np.all(np.abs(c1-c2)<1e-10))

True
False
True
False


In [13]:
# Largest penalty
penalty_max = 2*np.max(np.abs(X_data.T@(y_data-np.mean(y_data))))
penalty_max

1573.7251062735718

In [55]:
compute_objective(X_data, y_data, w_exact, b=np.mean(eps), penalty=1)

501.5697052633708

In [100]:
lasso_coordinate_descent(X_data,y_data,np.zeros(dim_d),penalty=10,tol=1e-3, debug=True)

obj: 2005.6044035387117, w: 0.6257519289028761
obj: 2590.4157252592095, w: 0.22052426422194032
obj: 2848.4235999562597, w: 0.05809234589304357
obj: 2921.158368887499, w: 0.012976314026921953
obj: 2932.7731366543617, w: 0.0063739038678709
obj: 2930.032650571152, w: 0.0026161300979966606
obj: 2926.4258522450973, w: 0.00144606504744621
obj: 2924.52743655298, w: 0.0005216653520096481


(array([-2.82564273e-02, -1.80425259e-02,  4.39743626e-02,  3.66105736e-02,
        -2.84474045e-02,  6.08447205e-02, -3.99019241e-03,  2.59845911e-02,
         7.92262580e-02, -1.55315382e-02,  1.17467128e-01,  0.00000000e+00,
         3.77863025e-02,  1.03235365e-01,  3.07892147e-02,  1.69865967e-02,
         3.34290929e-02,  1.85784047e-01,  7.79189105e-02,  7.00947450e-02,
         9.54474237e-02,  1.59204099e-01,  8.72815268e-02,  5.16435756e-02,
         1.87752558e-01,  0.00000000e+00,  1.55551953e-01,  1.68990270e-01,
         6.60761839e-02,  1.82205076e-01,  6.38102594e-02,  6.91493031e-02,
         1.96770920e-01,  8.67274410e-02,  2.15896303e-01,  1.32144007e-01,
         1.44938725e-01,  1.39609535e-01,  5.14590606e-02,  1.34091711e-01,
         2.24838007e-01,  1.41141052e-01,  1.32894147e-01,  1.61906401e-01,
         1.07625111e-01,  9.80939310e-02,  2.86813084e-01,  1.49801710e-01,
         8.64662147e-02,  2.00690811e-01,  2.20447832e-01,  1.26515084e-01,
         1.0

In [92]:
coordinate_descent(X_data, y_data, w=w0, penalty=1, delta_stop=1e-3, max_iterations=20)

obj: 235681.66508461637, w: 1.8319378666448298
obj: 743167.9052174543, w: 2.6370556517805808
obj: 2831915.3301122114, w: 4.795879537328176
obj: 12038162.852387473, w: 10.27519016355577
obj: 54560224.67521568, w: 22.27825102312328


KeyboardInterrupt: 

## This zone is for playing

In [95]:
a = np.ma.array([1,1,1], mask=[0,1, 0])
b = np.array([[1,2,3],[4,5, 6]])
c = np.array([-1,0,1])

In [100]:
np.ma.dot(b,a)

masked_array(data=[4, 10],
             mask=[False, False],
       fill_value=999999)

In [25]:
np.ma.dot(b,a,strict=True)

masked_array(data=[4, 10],
             mask=[False, False],
       fill_value=999999)

In [53]:
test = np.ma.array(np.tile(c,(3,1)).T, mask=np.identity(3))

In [57]:
np.ma.dot(b,test)

masked_array(
  data=[[3, 2, -1],
        [6, 2, -4]],
  mask=[[False, False, False],
        [False, False, False]],
  fill_value=999999)

In [58]:
np.dot(b,test)

masked_array(
  data=[[2, 2, 2],
        [2, 2, 2]],
  mask=False,
  fill_value=999999)

In [93]:
M1 = np.ones([10,3]).T
M1[0,:]=-1
M1[1,:]=0
print(M1.shape)
M1

(3, 10)


array([[-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]])

In [94]:
M2 = np.zeros([10,3])
M2[0,0]=1
M2[0,1]=2
M2[0,2]=3
print(M2.shape)
M2

(10, 3)


array([[1., 2., 3.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [95]:
np.dot(M1,M2)

array([[-1., -2., -3.],
       [ 0.,  0.,  0.],
       [ 1.,  2.,  3.]])

In [99]:
np.einsum('ij,ji->i',M1,M2)

array([-1.,  0.,  3.])

In [131]:
def test(x):
    if (x < 0).any():
        raise Exception('Test fail')
    x[0]=2
    return x

In [132]:
x = np.copy(c)
print(c)
test(x)
x

[-1  0  1]


Exception: Test fail

In [135]:
100 < np.inf

True

In [139]:
d,_ = b.shape

In [144]:
np.tile(w_exact,(10,1)).T.shape

(1000, 10)

In [ ]:
#     case1 = c < -penalty
#     case2 = (-penalty <= c) & (c <= penalty)
#     case3 = penalty < c
    
#     if case1.any():
#         w_new[case1] = np.divide(c[case1]+penalty, a[case1])
#     if case2.any():
#         w_new[case2] = 0
#     if case3.any():
#         w_new[case3] = np.divide(c[case3]-penalty, a[case3])